##### **Code for calculating the P1, P2, and Total proportion of time the ant interacts with the larva. The duration of the P1 phase is also recorded.**
##### This script requires the files 'timedata_C*.csv', 'TransitionPoints.csv' and 'stats_corrected.csv' resulting from Code_1, Code_3, and manual correction of 'larva_detect_frame' if required.

##### The output of this code are the measures plotted and used for statistical analyses of the brood care behavior assay in various experiments in Paul et al. 2025.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from natsort import natsorted

#define path to the data files
path = '/Users/Alex/Desktop/Broodcare_assay_Code_&_Sample_data/antrax/analysis/'
#define path to save the results
save_path = '/Users/Alex/Desktop/Broodcare_assay_Code_&_Sample_data/antrax/analysis/'

#load the Transition Points
transpoints = np.loadtxt(path + 'TransitionPoints.csv', delimiter=',', skiprows=1, usecols = 1)

#load the True Larva Detection frames for each ant from the 'stats_corrected.csv' file
trueLarvaDet = np.loadtxt(path + 'stats_corrected.csv', delimiter=',', skiprows=1, usecols = 7)

#load the timedata_C*.csv files and then sort them into natural order
fnames = glob.glob(path + 'timedata_C*.csv')
sfnames = natsorted(fnames)

#initialize numpy arrays to store the results
P1 = np.array([])
P2 = np.array([])
Total = np.array([])

#creates an empty array to store the Ant id
ant_id = np.array([])

i=0
for x in sfnames:
    file = sfnames[i]

    #set the ant_id
    ant_id = np.append(ant_id, 'C'+str(i+1))

    #load the 'interacting' data from the current timedata_C*.csv file
    interacting = np.loadtxt(file, delimiter=',', skiprows=1, usecols = (5))

    # Calculate the proportion of time ant interacts with larva in phase P1 (P1 proportion time with larva)
    # Note: transpoints is recorded in TransitionPoints.csv as if trueLarvaDet is frame 0, so have to add tureLarvaDet to transpoints to get the correct frame number
    P1_prop_twl = np.average(interacting[int(trueLarvaDet[i]):(int(trueLarvaDet[i])+int(transpoints[i]))])
    P1 = np.append(P1, P1_prop_twl)
    
    #Calculate the proportion of time ant interacts with larva in phase P2 (P2 proportion time with larva)
    P2_prop_twl = np.average(interacting[(int(trueLarvaDet[i])+int(transpoints[i])):])
    P2 = np.append(P2, P2_prop_twl)
    
    #Calculate the total proportion of time ant interacts with larva from the moment of larva detection to the end of the assay (Total proportion time with larva)
    Total_prop_twl = np.average(interacting[int(trueLarvaDet[i]):])
    Total = np.append(Total, Total_prop_twl)
    
    i+=1

#create a dataframe of the ant_id, transpoints (converted to hours), P1, P2, and Total
df = pd.DataFrame
df = pd.DataFrame({'ant_id': ant_id, 'P1_duration (hours)': transpoints/18000, 'P1_prop_twl': P1, 'P2_prop_twl': P2, 'Total_prop_twl': Total})

#save the results to a csv file
df.to_csv(save_path + 'Results_P1dur_P1_P2_Total.csv', index=False)
